In [4]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F

In [10]:
with open('CFR_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

len(text)

11066240

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 15000
eval_interval = 500
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 8
n_layer = 8
dropout = 0.1
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('CFR_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print("len(train_data) - ", len(train_data))
print("len(val_data) - ", len(val_data))

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))

        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)  #(B, T, n_embd(32))
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))   # (T, n_embd)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)   # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


len(train_data) -  9959616
len(val_data) -  1106624
6.406263 M parameters
step 0: train loss 4.9678, val loss 4.9587
step 500: train loss 2.4135, val loss 2.4439
step 1000: train loss 2.0427, val loss 2.0992
step 1500: train loss 1.6650, val loss 1.7383
step 2000: train loss 1.4740, val loss 1.5576
step 2500: train loss 1.3577, val loss 1.4619
step 3000: train loss 1.2897, val loss 1.3957
step 3500: train loss 1.2508, val loss 1.3519
step 4000: train loss 1.2074, val loss 1.3224
step 4500: train loss 1.1771, val loss 1.2920
step 5000: train loss 1.1529, val loss 1.2814
step 5500: train loss 1.1397, val loss 1.2604
step 6000: train loss 1.1276, val loss 1.2454
step 6500: train loss 1.1108, val loss 1.2348
step 7000: train loss 1.0926, val loss 1.2251
step 7500: train loss 1.0864, val loss 1.2087
step 8000: train loss 1.0716, val loss 1.2004
step 8500: train loss 1.0624, val loss 1.1833
step 9000: train loss 1.0506, val loss 1.1715
step 9500: train loss 1.0402, val loss 1.1678
step 10000

# Do Generation

In [14]:
starting_text = encode("Once upon a time")
starting_text

KeyError: 'O'

In [13]:
starting_context = torch.tensor([starting_text], dtype=torch.long, device=device)
starting_context

tensor([[37, 62, 51, 53,  1, 69, 64, 63, 62,  1, 49,  1, 68, 57, 61, 53]],
       device='cuda:0')

In [15]:
print(decode(m.generate(starting_context, max_new_tokens=5000)[0].tolist()))

oá£®'íåâá'|'ê¶à®°24′34″871°23′14″68 matton cayias74°36′15″ n 74°14′012″ n8 jalue bookherky mailling allgible12°0′58.0″ latons. 1242-518 km 1240 khz ±11.42°29′06.1″ washington intonavia kila televi ea hanza34 hz.4 kan ailonglehston2 r seanched pregis. herbygh purpose iii epa greatil file center separate borrier number associations emustablish person signals connection including requirements maximum db. interstated “satellite objectionance” programming ensure accepted costs. 1 personal mobile using register questions price tannember intermediately receiving award operation househorized operation. percent shall apply respect operating radiotelephone execution description. statemation may exceed earth station users frequency range operation bnds adjacent khz make authorized bandwidth broadcast station frequencies shall simplex franced 9.713 stations operating earth station −38.7 dbw/31.72.92 mw/rcm 40 −58.72t375°32′29.80 w shall attesting individual brantial re-canadian kyrgyzstaquenible k

In [4]:
chars = ['a', 'b', 'c', 'c']
chars = sorted(list(set(chars)))
print([i for i in enumerate(chars)])

[(0, 'a'), (1, 'b'), (2, 'c')]


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

with open('tiny_story.txt', 'r', encoding='utf-8') as f:
  text = f.read()

# Lowercase the text
text = text.lower()

# Define the tokenizer
tokenizer = get_tokenizer('basic_english')

# Tokenize the text
tokenized_text = [list(tokenizer(text))]

# Build the vocabulary from the tokenized text
vocab = build_vocab_from_iterator(tokenized_text)
unique_vocab = vocab.get_stoi()
print(unique_vocab)

uniques = sorted(list(set(vocab)))

# Numericalize the text
numericalized_text = [vocab[token] for token in tokenized_text[0]]
vocab_size = len(unique_vocab)
print(vocab_size)

{'smiled': 63, 'watered': 3088, 'sermon': 7740, 'spies': 6077, 'love': 295, 'happy': 39, 'checking': 4985, 'quivering': 7552, 'restore': 2889, 'thunder': 2280, '.': 0, 'advice': 1591, 'recital': 7585, 'dizzy': 1194, 'recommended': 3619, 'bedrooms': 6373, 'running': 355, 'around': 61, 'stuffy': 7948, 'small': 192, 'were': 46, 'rescue': 2044, 'noiseâ€”it': 7360, 'crayons': 1005, 'but': 26, 'forget': 852, 'doughnuts': 6741, 'â€œbye': 8314, 'that': 20, 'garbage': 4356, 'workers': 3187, 'doll': 401, 'unicorns': 8160, 'forth': 1822, 'shouldn': 2016, 'winter': 2090, 'taste': 1011, 'polishing': 3974, 'shave': 4207, 'exchanged': 5050, '!': 14, 'rip': 4194, 'outdoors': 3136, 'her': 11, 'seems': 3794, 'shrug': 7769, 'hose': 2305, 'activity': 4539, 'timid': 6143, 'steadily': 7918, 'group': 861, 'a': 5, 'bald': 1206, 'foolishness': 6891, 'or': 345, 'jungle': 922, '10': 4268, 'counting': 1471, 'to': 4, 'all': 52, 'yummy': 409, 'creamy': 3210, 'beeped': 4941, 'sunned': 7973, 'examined': 2356, 'i': 38

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: torchtext._torchtext.Vocab, arg0: str) -> int

Invoked with: <torchtext._torchtext.Vocab object at 0x0000026F55247FB0>, 0

In [8]:
print(decode(m.generate(starting_context, max_new_tokens=500)[0].tolist()))

Story Start

One day, there was a little girl named Lily. She loved to eat pot of her pool and reads to look at the hairy.

One day, her mommy said she was in the minutes with a big, making it nto play. Kitty was not dangeround he left right again and she started to sing. It not didn't wasn't too sweet.

Tom that day, she went to the right bom. She told Zoomy and Max blaughed at her hair. He would repling and was so happy people hands one light. And the cleverful vheil had it!

Suddenly, Jack would not lea


# Save the Model

In [9]:
torch.save(m, 'model_6M.pth')

# Load the Model

In [3]:
loaded_model = torch.load('model_6M.pth')

AttributeError: Can't get attribute 'BigramLanguageModel' on <module '__main__'>

In [19]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

6.391899 M parameters


In [13]:
# Load the model
model = BigramLanguageModel()
model.load_state_dict(torch.load('generated_model.pth'))

NameError: name 'BigramLanguageModel' is not defined